## Canvas Map

Creates an hexagon map as a base for all research and development

Uber hexagon project:
https://nbviewer.jupyter.org/github/uber/h3-py-notebooks/blob/master/H3%20API%20examples%20on%20Urban%20Analytics.ipynb

In [4]:
%%sh
cat <<EOF > requirements.txt
h3==3.1.0
pandas>=0.23
geojson==2.4.1
folium==0.7.0
seaborn==0.9.0
area==1.1.1
statsmodels==0.9.0
shapely
descartes
geopandas
EOF

In [7]:
%%sh
pip3 install -r requirements.txt

Process is interrupted.


In [63]:
%%sh
pip3 install descartes

In [9]:
import json
import pandas as pd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from shapely.geometry import Point, Polygon
import geopandas as gpd
from folium import Map, Marker, GeoJson

In [2]:
with open("convertcsv.geojson") as f:
    loc_geodata = json.load(f)
    
df_subzones = pd.DataFrame(pd.json_normalize(loc_geodata['features']))
df_subzones.head(5)

,type,geometry.type,geometry.coordinates,properties.localidad
0,Feature,Point,"[-74.0571685, 4.6869486]",usaquen
1,Feature,Point,"[-74.0563848, 4.6867864]",usaquen
2,Feature,Point,"[-74.0549382, 4.6864427]",usaquen
3,Feature,Point,"[-74.0546701, 4.6863871]",usaquen
4,Feature,Point,"[-74.053883, 4.6862325]",usaquen


In [3]:
df = df_subzones.groupby('properties.localidad')['geometry.coordinates'].apply(lambda x: Polygon(x.tolist())).reset_index()

In [4]:
df

,properties.localidad,geometry.coordinates
0,antonio narino,"POLYGON ((-74.1308846 4.5936364, -74.130426099..."
1,barrios unidos,"POLYGON ((-74.0933105 4.6662185, -74.0929208 4..."
2,bosa,"POLYGON ((-74.22356689999999 4.6255475, -74.22..."
3,chapinero,"POLYGON ((-74.0683599 4.6288107, -74.067597500..."
4,ciudad bolivar,"POLYGON ((-74.2122251 4.3916744, -74.2103292 4..."
5,engativa,"POLYGON ((-74.16044890000001 4.7201089, -74.15..."
6,fontibon,"POLYGON ((-74.1769551 4.7023652, -74.1763543 4..."
7,kennedy,"POLYGON ((-74.1855841 4.6466405, -74.185573700..."
8,la candelaria,"POLYGON ((-74.0821845 4.5932599, -74.0814962 4..."
9,los martires,"POLYGON ((-74.10704699999999 4.5984312, -74.10..."


In [20]:
#Delete Sumapaz, usme, Ciudad Bolivar y san cristobal because are big locations without much interest population
df = df.drop([4, 12, 15, 19])
df

KeyError: '[12 15 19] not found in axis'

In [78]:
from shapely.ops import cascaded_union
polygons = [df['geometry.coordinates'][0], df['geometry.coordinates'][1],
           df['geometry.coordinates'][2], df['geometry.coordinates'][3],
           df['geometry.coordinates'][5], df['geometry.coordinates'][6], 
           df['geometry.coordinates'][7], df['geometry.coordinates'][8], 
           df['geometry.coordinates'][9], df['geometry.coordinates'][10], 
           df['geometry.coordinates'][11], df['geometry.coordinates'][13], 
           df['geometry.coordinates'][14], df['geometry.coordinates'][16], 
           df['geometry.coordinates'][17], df['geometry.coordinates'][18]]
boundary = gpd.GeoSeries(cascaded_union(polygons))
bog_bound = boundary.boundary.to_json()
bog_bound

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "MultiLineString", "coordinates": [[[-74.0945836, 4.5757791], [-74.0948332, 4.5754841], [-74.0955184, 4.5747504], [-74.0967297, 4.5734452], [-74.0970094, 4.5731579], [-74.097249, 4.5728979], [-74.0977261, 4.5723958], [-74.0979954, 4.5721126], [-74.098121, 4.5719838], [-74.0982829, 4.5718168], [-74.0989757, 4.5710402], [-74.0991673, 4.5708158], [-74.0994881, 4.5704226], [-74.0998887, 4.5699449], [-74.1005997, 4.5691018], [-74.1009588, 4.5686612], [-74.1020254, 4.5674993], [-74.1026571, 4.5668513], [-74.1035211, 4.5658403], [-74.1038185, 4.5655273], [-74.1042708, 4.5650279], [-74.1050668, 4.5645019], [-74.1056364, 4.5642174], [-74.1059735, 4.5640712], [-74.1063562, 4.563933], [-74.1055024, 4.5626312], [-74.1053148, 4.5623746], [-74.1051237, 4.5621639], [-74.104619, 4.5616774], [-74.1042346, 4.561153], [-74.103921, 4.5606986], [-74.1036487, 4.5602613], [-74.1035146, 4.5598433]

In [76]:
map_bogota = Map(location= [4.6736789,-74.0400262], zoom_start=10, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
GeoJson(
        bog_bound,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'green',
            'weight': 3,
            'fillOpacity': 0
        }, 
        name = "Subzone" 
    ).add_to(map_bogota)
map_bogota.save('map_bogota.html')
map_bogota

### Fill Bogota with hexagons

In [79]:
geoJson = json.loads(bog_bound)
geoJson['features'][0]['geometry']['type'] = 'Polygon'
geoJson['features'][0]['geometry']

{'type': 'Polygon',
 'coordinates': [[[-74.0945836, 4.5757791],
   [-74.0948332, 4.5754841],
   [-74.0955184, 4.5747504],
   [-74.0967297, 4.5734452],
   [-74.0970094, 4.5731579],
   [-74.097249, 4.5728979],
   [-74.0977261, 4.5723958],
   [-74.0979954, 4.5721126],
   [-74.098121, 4.5719838],
   [-74.0982829, 4.5718168],
   [-74.0989757, 4.5710402],
   [-74.0991673, 4.5708158],
   [-74.0994881, 4.5704226],
   [-74.0998887, 4.5699449],
   [-74.1005997, 4.5691018],
   [-74.1009588, 4.5686612],
   [-74.1020254, 4.5674993],
   [-74.1026571, 4.5668513],
   [-74.1035211, 4.5658403],
   [-74.1038185, 4.5655273],
   [-74.1042708, 4.5650279],
   [-74.1050668, 4.5645019],
   [-74.1056364, 4.5642174],
   [-74.1059735, 4.5640712],
   [-74.1063562, 4.563933],
   [-74.1055024, 4.5626312],
   [-74.1053148, 4.5623746],
   [-74.1051237, 4.5621639],
   [-74.104619, 4.5616774],
   [-74.1042346, 4.561153],
   [-74.103921, 4.5606986],
   [-74.1036487, 4.5602613],
   [-74.1035146, 4.5598433],
   [-74.103331

In [80]:
from h3 import h3

set_hexagons = h3.polyfill(geo_json = geoJson['features'][0]['geometry'], res = 9)
list_hexagons = list(set_hexagons)
print("the subzone was filled with ", len(list_hexagons), "hexagons at resolution 9")

the subzone was filled with  930 hexagons at resolution 9


### Get latitud and logitud of a single hexagon

In [81]:
def reverse_lat_lon(hex_coords):
    geom_hex = []
    for lat_lon in hex_coords:
        geom_hex.append([lat_lon[1],lat_lon[0]])
        
    return geom_hex

In [82]:
one_hex_of_fill =  list_hexagons[0]
one_hex_of_fill_coords_latlon = h3.h3_to_geo_boundary(h3_address=one_hex_of_fill,geo_json=False)
one_hex_of_fill_coords_lonlat = reverse_lat_lon(hex_coords = one_hex_of_fill_coords_latlon)

print(one_hex_of_fill)
print(one_hex_of_fill_coords_latlon)
print(one_hex_of_fill_coords_lonlat)

89e76d61a2bffff
[[-74.09054773998108, 4.626910412313827], [-74.08859899863998, 4.627224728361666], [-74.08752900005143, 4.621291161456808], [-74.08840756377846, 4.615042606657816], [-74.09035630547079, 4.614726804669314], [-74.09142648310666, 4.620661043403171]]
[[4.626910412313827, -74.09054773998108], [4.627224728361666, -74.08859899863998], [4.621291161456808, -74.08752900005143], [4.615042606657816, -74.08840756377846], [4.614726804669314, -74.09035630547079], [4.620661043403171, -74.09142648310666]]


### Draw Bogota's Map with hexagons

In [88]:
from geojson.feature import *
import folium

def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already, along with the columns hex_id and value '''
    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

In [92]:
df_fill_hex = pd.DataFrame({"hex_id": list_hexagons})
df_fill_hex["value"] = 0
df_fill_hex['geometry'] = df_fill_hex.hex_id.apply(lambda x: 
                                                       {    "type" : "Polygon",
                                                             "coordinates": 
                                                   [h3.h3_to_geo_boundary(h3_address=x,geo_json=False)]
                                                        }
                                                    )


geojson_hx = hexagons_dataframe_to_geojson(df_fill_hex)

GeoJson(
        geojson_hx,
        style_function=lambda feature: {
            'fillColor': None,
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0
        }, 
        name = "Hexagons" 
    ).add_to(map_bogota)

folium.map.LayerControl('bottomright', collapsed=False).add_to(map_bogota)


map_bogota.save('map_bogota.html')
map_bogota